In [1]:
import pandas as pd

In [2]:
df_msr = pd.read_csv('../MSR/MSR_data_cleaned.csv')

/Users/mahbubcseju/opt/anaconda3/envs/codebert/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_devign = pd.read_json('../devign/function.json')

In [4]:
df_msr_ = []
for index, row in df_msr.iterrows():
    df_msr_.append(row)

In [5]:
df_devign_ = []
for index, row in df_devign.iterrows():
    df_devign_.append(row)

In [6]:
df_msr_[0]

Unnamed: 0                                                                      0
Access Gained                                                                None
Attack Origin                                                              Remote
Authentication Required                                             Single system
Availability                                                              Partial
CVE ID                                                              CVE-2015-8467
CVE Page                            https://www.cvedetails.com/cve/CVE-2015-8467/
CWE ID                                                                    CWE-264
Complexity                                                                 Medium
Confidentiality                                                           Partial
Integrity                                                                 Partial
Known Exploits                                                                NaN
Publish Date    

In [27]:
# df = pd.read_csv('fold_0_holdout.csv')

In [28]:
# df

,Unnamed: 0,dataset,example_index,project,commit_id,split
0,37,MSR,37,php,c351b47ce85a3a147cfa801fa9f0149ab4160834,holdout
1,38,MSR,38,php,c351b47ce85a3a147cfa801fa9f0149ab4160834,holdout
2,39,MSR,39,php,c351b47ce85a3a147cfa801fa9f0149ab4160834,holdout
3,40,MSR,40,php,c351b47ce85a3a147cfa801fa9f0149ab4160834,holdout
4,41,MSR,41,php,c351b47ce85a3a147cfa801fa9f0149ab4160834,holdout
...,...,...,...,...,...,...
10026,183337,MSR,183337,redis,1eb08bcd4634ae42ec45e8284923ac048beaa4c3,holdout
10027,183338,MSR,183338,redis,1eb08bcd4634ae42ec45e8284923ac048beaa4c3,holdout
10028,183339,MSR,183339,redis,52a00201fca331217c3b4b8b634f6a0f57d6b7d3,holdout
10029,183348,MSR,183348,rpm,f2d3be2a8741234faaa96f5fd05fdfdc75779a79,holdout


In [7]:
import json

In [8]:
def get_ind_data(row):
    if row['dataset'] == 'MSR':
        return json.dumps({
            'func': df_msr_[row['example_index']]['func_before'],
            'target': df_msr_[row['example_index']]['vul'],
            'dataset': row['dataset'],
            'idx': row['example_index'],
            'project': row['project']
        }) +"\n"
    elif row['dataset'] == 'devign':
        return json.dumps({
            'func': df_devign_[row['example_index']]['func'],
            'target': df_devign_[row['example_index']]['target'],
            'dataset': row['dataset'],
            'idx': row['example_index'],
            'project': row['project']
        }) +"\n"
    assert False

In [9]:
def get_data(df):
    train = []
    test = []
    valid = []
    holdout = []
    for ind, row in df.iterrows():
        if row['split'] == 'train':
            train.append(get_ind_data(row))
        if row['split'] == 'test':
            test.append(get_ind_data(row))
        if row['split'] == 'valid':
            valid.append(get_ind_data(row))
        if row['split'] == 'holdout':
            holdout.append(get_ind_data(row))
    return train, test, valid, holdout

In [10]:
def writejsonl(data, file_path):
    f = open(file_path, 'w')
    f.writelines(data)
    f.close()

In [11]:
import os
import shutil

In [16]:
for file in os.listdir("."):
    if file[-3:] == 'csv':
        folder = file[:-4]
        if not os.path.exists(folder):
            os.mkdir(folder)
        else:
            shutil.rmtree(folder)
            os.mkdir(folder)
        dataset = pd.read_csv(file)
        result = get_data(dataset)
        writejsonl(result[0], '{}/train.jsonl'.format(folder))
        writejsonl(result[1], '{}/test.jsonl'.format(folder))
        writejsonl(result[2], '{}/valid.jsonl'.format(folder))
        writejsonl(result[3], '{}/holdout.jsonl'.format(folder))

In [64]:
# for i in range(5):
#     dataset = pd.read_csv('fold_{}_dataset.csv'.format(i))
#     holdout = pd.read_csv('fold_{}_holdout.csv'.format(i))
#     result = [[], [], [], []]
#     res = get_data(dataset)
#     for j in range(4):
#         result[j] += res[j]
#     res = get_data(holdout)
#     for j in range(4):
#         result[j] += res[j]
#     if not os.path.exists('fold_{}'.format(i)):
#         os.mkdir('fold_{}'.format(i))
#     else:
#         shutil.rmtree('fold_{}'.format(i))
#         os.mkdir('fold_{}'.format(i))
#     writejsonl(result[0], 'fold_{}/train.jsonl'.format(i))
#     writejsonl(result[1], 'fold_{}/test.jsonl'.format(i))
#     writejsonl(result[2], 'fold_{}/valid.jsonl'.format(i))
#     writejsonl(result[3], 'fold_{}/holdout.jsonl'.format(i))
    
    